<a href="https://colab.research.google.com/github/tejeshbhalla/Machine-Learning-From-Scratch/blob/master/Lora%20From%20Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [368]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [398]:
class Mnist_Trainer(nn.Module):
   def __init__(self, init_dim, hidden_size, class_objects,dropout):
     super().__init__()
     self.hidden_size=hidden_size
     self.c=class_objects
     self.l1=nn.Linear(init_dim, hidden_size)
     self.relu=nn.ReLU()
     self.dropout=nn.Dropout(dropout)
     self.l2=nn.Linear(hidden_size, hidden_size)
     self.l3=nn.Linear(hidden_size, class_objects)


   def forward(self,x):
     x = self.l1(x)
     x = self.relu(x)
     x = self.dropout(x)

     x = self.l2(x)
     x = self.relu(x)
     x = self.dropout(x)

     x=self.l3(x)

     x=F.log_softmax(x,dim=-1)

     return x


def save_model(model):
  torch.save(model.state_dict(),'model.pt')



In [399]:
data=pd.read_csv('/content/sample_data/mnist_train_small.csv',header=None)

In [400]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [401]:
from torch.utils.data import Dataset

In [402]:
class DataFrameDataset(Dataset):
     def __init__(self, x,labels):
         self.x = x
         self.labels=labels

     def __len__(self):
         return len(self.x)

     def __getitem__(self, index):
         x_small=self.x[index]
         label=self.labels[index]
         return x_small, label

In [414]:
labels=data[0].values
x=data.values[:,1:]
mask = np.logical_and(labels >= 0, labels <= 4)
x=x[mask]
labels=labels[mask]

X_train,X_test,y_train,y_test=train_test_split(x,labels)


In [415]:
dataset=DataFrameDataset(X_train,y_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32)

dataset=DataFrameDataset(X_test,y_test)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32)

In [416]:
for batch in test_loader:
  print(batch[0],batch[1])
  break

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]) tensor([4, 2, 2, 2, 1, 0, 3, 1, 1, 4, 1, 2, 0, 1, 3, 0, 0, 0, 2, 4, 1, 3, 1, 3,
        0, 4, 0, 2, 0, 0, 1, 0])


In [417]:
model_1=Mnist_Trainer(784,64,5,0.3)
criterion = F.nll_loss

 # Optimizer
optimizer = torch.optim.Adam(model_1.parameters(), lr=0.001)

In [418]:
for m in model_1.parameters():
  print(m.numel())

50176
64
4096
64
320
5


In [419]:
def train_model(model,train_loader,val_loader,epochs=10,lora=False):
  last_val=-99999
  for epoch in range(epochs):
    train_accuracys=[]
    for batch in train_loader:
      inputs = batch[0].float()
      targets = batch[1]

      if lora:
        targets=targets-5


      optimizer.zero_grad()
      # Forward pass
      outputs = model(inputs)
      train_accuracy=(torch.argmax(outputs,dim=-1)==targets).sum()/targets.shape[0]
      train_accuracys.append(train_accuracy)
      # Loss
      loss = criterion(outputs, targets)
      # Backward pass
      loss.backward()

      # Update weights
      optimizer.step()


    val_losses = []
    val_accuracys=[]
    model.eval()  # Set model to eval mode
    with torch.no_grad():
      for val_batch in test_loader:
        val_inputs = val_batch[0].float()
        val_targets = val_batch[1]

        if lora:
          val_targets=val_targets-5

        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_targets)
        val_losses.append(val_loss.item())
        val_accuracy=(torch.argmax(val_outputs,dim=-1)==val_targets).sum()/val_targets.shape[0]
        val_accuracys.append(val_accuracy)
    if last_val< np.mean(val_accuracys):
        if not lora:
          save_model(model)
        last_val=np.mean(val_accuracys)
    # Print losses
    print(f'Epoch {epoch+1} - Train loss: {loss.item():.4f}, Val loss: {np.mean(val_losses):.4f}, Val Acc: {np.mean(val_accuracys)}, Train Acc: {np.mean(train_accuracys)}')

    # Set model back to train mode
    model.train()
  return val_accuracys,train_accuracys


In [420]:
acc_val,acc_train=train_model(model_1,train_loader,test_loader,10)


Epoch 1 - Train loss: 0.3894, Val loss: 0.1607, Val Acc: 0.959765613079071, Train Acc: 0.8252888321876526
Epoch 2 - Train loss: 0.2070, Val loss: 0.1312, Val Acc: 0.9638393521308899, Train Acc: 0.9213234782218933
Epoch 3 - Train loss: 0.0076, Val loss: 0.1246, Val Acc: 0.9650112390518188, Train Acc: 0.939994752407074
Epoch 4 - Train loss: 0.0703, Val loss: 0.1201, Val Acc: 0.971875011920929, Train Acc: 0.9518119692802429
Epoch 5 - Train loss: 0.0571, Val loss: 0.1028, Val Acc: 0.9724330902099609, Train Acc: 0.9585084319114685
Epoch 6 - Train loss: 0.0765, Val loss: 0.1075, Val Acc: 0.973214328289032, Train Acc: 0.9615283608436584
Epoch 7 - Train loss: 0.0854, Val loss: 0.1092, Val Acc: 0.9716518521308899, Train Acc: 0.9664390683174133
Epoch 8 - Train loss: 0.0659, Val loss: 0.1101, Val Acc: 0.9730468988418579, Train Acc: 0.9708508253097534
Epoch 9 - Train loss: 0.1197, Val loss: 0.1007, Val Acc: 0.9765625, Train Acc: 0.9714285731315613
Epoch 10 - Train loss: 0.0509, Val loss: 0.1055, V

In [421]:
#Lora Experiment

In [448]:
class Mnist_Trainer_Lora(nn.Module):
   def __init__(self, init_dim, hidden_size, class_objects,dropout,lora_rank,lora_alpha):
     super().__init__()
     self.hidden_size=hidden_size
     self.c=class_objects
     self.lora_rank=lora_rank
     self.lora_alpha=lora_alpha

     self.l1=nn.Linear(init_dim, hidden_size)
     self.l2=nn.Linear(hidden_size, hidden_size)
     self.l3=nn.Linear(hidden_size, class_objects)
     self.relu=nn.ReLU()
     self.dropout=nn.Dropout(dropout)

     #lora init
     self.lora_l1_A=nn.Parameter(torch.empty(init_dim,lora_rank))
     self.lora_l1_B=nn.Parameter(torch.empty(lora_rank,hidden_size))

     self.lora_l2_A=nn.Parameter(torch.empty(hidden_size,lora_rank))
     self.lora_l2_B=nn.Parameter(torch.empty(lora_rank,hidden_size))

     self.lora_l3_A=nn.Parameter(torch.empty(hidden_size,lora_rank))
     self.lora_l3_B=nn.Parameter(torch.empty(lora_rank,class_objects))


     for n,p in self.named_parameters():
        if 'lora' in n:

          if n[-1]=='A':
            nn.init.kaiming_uniform_(p,a=math.sqrt(5))
          else:
            nn.init.zeros_(p)
        if 'lora' not in n:
          p.requires_grad=False


   def compute_lora(self,x,layer,A,B):
     h=layer(x)
     h+=x@(A@B)*self.lora_alpha
     return h

   def forward(self,x):
     x=self.compute_lora(x,self.l1,self.lora_l1_A,self.lora_l1_B)
     x=self.relu(x)
     x=self.dropout(x)

     x=self.compute_lora(x,self.l2,self.lora_l2_A,self.lora_l2_B)
     x=self.relu(x)
     x=self.dropout(x)

     x=self.compute_lora(x,self.l3,self.lora_l3_A,self.lora_l3_B)

     x=F.log_softmax(x,dim=-1)
     return x



def save_model(model):
  torch.save(model.state_dict(),'model.pt')

def lora_experiment(rank,train_loader,test_loader):
  state_dict=torch.load('model.pt')
  model=Mnist_Trainer_Lora(784,64,5,0.1,rank,1)
  model.load_state_dict(state_dict,strict=False)
  train_model(model,train_loader,test_loader,10,True)
  return model



In [440]:
labels=data[0].values
x=data.values[:,1:]
mask = np.logical_and(labels >= 5, labels <= 9)
x=x[mask]
labels=labels[mask]

X_train,X_test,y_train,y_test=train_test_split(x,labels)

dataset=DataFrameDataset(X_train,y_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32)

dataset=DataFrameDataset(X_test,y_test)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32)

In [450]:
model=lora_experiment(64,train_loader,test_loader)

Epoch 1 - Train loss: 4.0175, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.3131735026836395
Epoch 2 - Train loss: 3.8863, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31303879618644714
Epoch 3 - Train loss: 2.9128, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31492456793785095
Epoch 4 - Train loss: 3.9416, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.3131735026836395
Epoch 5 - Train loss: 3.7393, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31128770112991333
Epoch 6 - Train loss: 3.5754, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31505927443504333
Epoch 7 - Train loss: 2.3931, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31290408968925476
Epoch 8 - Train loss: 3.5544, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.3173491358757019
Epoch 9 - Train loss: 3.8735, Val loss: 4.3658, Val Acc: 0.3337339758872986, Train Acc: 0.31384697556495667
Epoch 10 - Train loss: 3.8469, 

In [427]:
labels

array([6, 5, 7, ..., 5, 9, 5])

In [428]:
state_dict=torch.load('model.pt')

In [429]:
state_dict

OrderedDict([('l1.weight',
              tensor([[-0.0355, -0.0180, -0.0316,  ...,  0.0088, -0.0091,  0.0273],
                      [-0.0257, -0.0146,  0.0088,  ...,  0.0316,  0.0209, -0.0250],
                      [ 0.0010, -0.0340,  0.0038,  ..., -0.0244, -0.0078,  0.0073],
                      ...,
                      [ 0.0188,  0.0176,  0.0160,  ..., -0.0348,  0.0287,  0.0354],
                      [-0.0117,  0.0017, -0.0316,  ..., -0.0096, -0.0087,  0.0164],
                      [-0.0005, -0.0185, -0.0138,  ..., -0.0085, -0.0073, -0.0336]])),
             ('l1.bias',
              tensor([-0.0305, -0.0323, -0.0414,  0.0064, -0.0640,  0.0104, -0.0009,  0.0156,
                      -0.0369, -0.0342, -0.0201,  0.0142,  0.0265,  0.0282, -0.0343,  0.0258,
                      -0.0470, -0.0100, -0.0235, -0.0371, -0.0202, -0.0212, -0.0550, -0.0284,
                      -0.0101, -0.0315, -0.0057, -0.0463, -0.0005, -0.0155, -0.0541,  0.0220,
                      -0.0362, -0.0016

In [449]:
state_dict=torch.load('model.pt')
model=Mnist_Trainer_Lora(784,64,5,0.1,4,1)
model.load_state_dict(state_dict,strict=False)

_IncompatibleKeys(missing_keys=['lora_l1_A', 'lora_l1_B', 'lora_l2_A', 'lora_l2_B', 'lora_l3_A', 'lora_l3_B'], unexpected_keys=[])

In [451]:
logits=model(batch[0].float())

In [452]:
torch.argmax(logits,dim=-1)

tensor([4, 2, 2, 2, 1, 0, 3, 1, 1, 4, 1, 2, 0, 1, 3, 0, 0, 0, 2, 4, 1, 3, 1, 3,
        0, 4, 0, 2, 0, 0, 1, 0])

In [453]:
batch[1]

tensor([4, 2, 2, 2, 1, 0, 3, 1, 1, 4, 1, 2, 0, 1, 3, 0, 0, 0, 2, 4, 1, 3, 1, 3,
        0, 4, 0, 2, 0, 0, 1, 0])